# 0. Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from tqdm.notebook import tqdm
from tqdm import tqdm_notebook
import classifiers
import importlib
import json
import warnings
warnings.filterwarnings("ignore")
import pickle

In [ ]:
%%javascript
(function(on) {
const e=$( "<a>Setup failed</a>" );
const ns="js_jupyter_suppress_warnings";
var cssrules=$("#"+ns);
if(!cssrules.length) cssrules = $("<style id='"+ns+"' type='text/css'>div.output_stderr { } </style>").appendTo("head");
e.click(function() {
    var s='Showing';  
    cssrules.empty()
    if(on) {
        s='Hiding';
        cssrules.append("div.output_stderr, div[data-mime-type*='.stderr'] { display:none; }");
    }
    e.text(s+' warnings (click to toggle)');
    on=!on;
}).click();
$(element).append(e);
})(true);

# 1. Data Load

In [ ]:
process_hypergeometric = np.array(pd.read_csv("../../data/processed/metrics/process_hypergeometric_fp.csv", sep=',', header=0, index_col=0).transpose())
process_closeness = np.array(pd.read_csv("../../data/processed/metrics/process_closeness_fp.csv", sep=',', header=0, index_col=0).transpose())
process_betweenness = np.array(pd.read_csv("../../data/processed/metrics/process_betweenness_fp.csv", sep=',', header=0, index_col=0).transpose())
process_fraction_betweenness = np.array(pd.read_csv("../../data/processed/metrics/process_fraction_betweenness_fp.csv", sep=',', header=0, index_col=0).transpose())
process_rwr = np.array(pd.read_csv("../../data/processed/metrics/process_rwr_fp.csv", sep=',', header=0, index_col=0).transpose())

In [ ]:
disease_hypergeometric = np.array(pd.read_csv("../../data/processed/metrics/disease_hypergeometric_fp.csv", sep=',', header=0, index_col=0).transpose())
disease_closeness = np.array(pd.read_csv("../../data/processed/metrics/disease_closeness_fp.csv", sep=',', header=0, index_col=0).transpose())
disease_betweenness = np.array(pd.read_csv("../../data/processed/metrics/disease_betweenness_fp.csv", sep=',', header=0, index_col=0).transpose())
disease_fraction_betweenness = np.array(pd.read_csv("../../data/processed/metrics/disease_fraction_betweenness_fp.csv", sep=',', header=0, index_col=0).transpose())
disease_rwr = np.array(pd.read_csv("../../data/processed/metrics/disease_rwr_fp.csv", sep=',', header=0, index_col=0).transpose())

In [ ]:
disease_hypergeometric_conservative = np.array(pd.read_csv("../../data/processed/metrics/disease_hypergeometric_conservative_fp.csv", sep=',', header=0, index_col=0).transpose())
disease_closeness_conservative = np.array(pd.read_csv("../../data/processed/metrics/disease_closeness_conservative_fp.csv", sep=',', header=0, index_col=0).transpose())
disease_betweenness_conservative = np.array(pd.read_csv("../../data/processed/metrics/disease_betweenness_conservative_fp.csv", sep=',', header=0, index_col=0).transpose())
disease_fraction_betweenness_conservative = np.array(pd.read_csv("../../data/processed/metrics/disease_fraction_betweenness_conservative_fp.csv", sep=',', header=0, index_col=0).transpose())
disease_rwr_conservative = np.array(pd.read_csv("../../data/processed/metrics/disease_rwr_conservative_fp.csv", sep=',', header=0, index_col=0).transpose())

In [ ]:
process_closeness_df = pd.read_csv("../../data/processed/metrics/process_closeness_fp.csv", sep=',', header=0, index_col=0)
process_labels_df = pd.read_csv("../../data/processed/reactome_labels_fp.csv", sep=',', names=process_closeness_df.columns)
process_labels_df['protein_id'] = process_closeness_df.index
process_labels_df.set_index('protein_id', inplace=True)

In [ ]:
hypergeometric_df = pd.read_csv("../../data/processed/metrics/disease_hypergeometric_fp.csv", sep=',', header=0, index_col=0)
disgenet_labels_df = pd.read_csv("../../data/processed/disgenet_labels_fp.csv", sep=',', names=hypergeometric_df.columns)
disgenet_labels_df['protein_id'] = hypergeometric_df.index
disgenet_labels_df.set_index('protein_id', inplace=True)

In [ ]:
hypergeometric_conservative_df = pd.read_csv("../../data/processed/metrics/disease_hypergeometric_conservative_fp.csv", sep=',', header=0, index_col=0)
disgenet_labels_conservative_df = pd.read_csv('../../data/processed/disgenet_conservative_labels_fp.csv', names=hypergeometric_conservative_df.columns)
disgenet_labels_conservative_df['protein_id'] = hypergeometric_conservative_df.index
disgenet_labels_conservative_df.set_index('protein_id', inplace=True)

In [ ]:
process_hypergeometric_fs = pd.read_csv("../../data/processed/fs/reactome_hypergeometric_fs_fp.csv", sep=',', header=0)
process_closeness_fs = pd.read_csv("../../data/processed/fs/reactome_closeness_fs_fp.csv", sep=',', header=0)
process_betweenness_fs = pd.read_csv("../../data/processed/fs/reactome_betweenness_fs_fp.csv", sep=',', header=0)
process_fraction_betweenness_fs = pd.read_csv("../../data/processed/fs/reactome_fraction_betweenness_fs_fp.csv", sep=',', header=0)
process_rwr_fs = pd.read_csv("../../data/processed/fs/reactome_rwr_fs_fp.csv", sep=',', header=0)

disease_hypergeometric_fs = pd.read_csv("../../data/processed/fs/disease/disease_hypergeometric_fs_fp.csv", sep=',', header=0)
disease_closeness_fs = pd.read_csv("../../data/processed/fs/disease/disease_closeness_fs_fp.csv", sep=',', header=0)
disease_betweenness_fs = pd.read_csv("../../data/processed/fs/disease/disease_betweenness_fs_fp.csv", sep=',', header=0)
disease_fraction_betweenness_fs = pd.read_csv("../../data/processed/fs/disease/disease_fraction_betweenness_fs_fp.csv", sep=',', header=0)
disease_rwr_fs = pd.read_csv("../../data/processed/fs/disease/disease_rwr_fs_fp.csv", sep=',', header=0)

disease_conservative_hypergeometric_fs = pd.read_csv("../../data/processed/fs/disease/disease_hypergeometric_fs_conservative_fp.csv", sep=',', header=0)
disease_conservative_closeness_fs = pd.read_csv("../../data/processed/fs/disease/disease_closeness_fs_conservative_fp.csv", sep=',', header=0)
disease_conservative_betweenness_fs = pd.read_csv("../../data/processed/fs/disease/disease_betweenness_fs_conservative_fp.csv", sep=',', header=0)
disease_conservative_fraction_betweenness_fs = pd.read_csv("../../data/processed/fs/disease/disease_fraction_betweenness_fs_conservative_fp.csv", sep=',', header=0)
disease_conservative_rwr_fs = pd.read_csv("../../data/processed/fs/disease/disease_rwr_fs_conservative_fp.csv", sep=',', header=0)

In [ ]:
process_hypergeometric_test = pd.read_csv("../../data/processed/fs/reactome_hypergeometric_test_fp.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
process_closeness_test = pd.read_csv("../../data/processed/fs/reactome_closeness_test_fp.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
process_betweenness_test = pd.read_csv("../../data/processed/fs/reactome_betweenness_test_fp.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
process_fraction_betweenness_test = pd.read_csv("../../data/processed/fs/reactome_fraction_betweenness_test_fp.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
process_rwr_test = pd.read_csv("../../data/processed/fs/reactome_rwr_test_fp.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1

disease_hypergeometric_test = pd.read_csv("../../data/processed/fs/disease/disease_hypergeometric_test_fp.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
disease_closeness_test = pd.read_csv("../../data/processed/fs/disease/disease_closeness_test_fp.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
disease_betweenness_test = pd.read_csv("../../data/processed/fs/disease/disease_betweenness_test_fp.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
disease_fraction_betweenness_test = pd.read_csv("../../data/processed/fs/disease/disease_fraction_betweenness_test_fp.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
disease_rwr_test = pd.read_csv("../../data/processed/fs/disease/disease_rwr_test_fp.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1

disease_conservative_hypergeometric_test = pd.read_csv("../../data/processed/fs/disease/disease_hypergeometric_test_conservative_fp.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
disease_conservative_closeness_test = pd.read_csv("../../data/processed/fs/disease/disease_closeness_test_conservative_fp.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
disease_conservative_betweenness_test = pd.read_csv("../../data/processed/fs/disease/disease_betweenness_test_conservative_fp.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
disease_conservative_fraction_betweenness_test = pd.read_csv("../../data/processed/fs/disease/disease_fraction_betweenness_test_conservative_fp.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1
disease_conservative_rwr_test = pd.read_csv("../../data/processed/fs/disease/disease_rwr_test_conservative_fp.csv", sep=',', header=0).transpose().to_numpy(dtype='int')-1

# 2. Classification Tasks

## 2.1. Complete Network

### 2.1.1. Process

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
hypergeometric_lgr_proba_clf, hypergeometric_cv, hypergeometric_n_fs, hypergeometric_models = classifiers.multiple_fs_classifier_fp(clf, parameters, process_hypergeometric, process_hypergeometric_test, process_hypergeometric_fs, process_labels_df, jobs=20)
hypergeometric_lgr_proba_clf.to_csv('../../models/false_positive/probability/process_hypergeometric_lgr_proba.csv', index=False)
with open('../../models/false_positive/cv_results/process_hypergeometric.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in hypergeometric_cv))
with open("../../models/false_positive/n_fs/process_hypergeometric.txt", "w") as f:
    for s in hypergeometric_n_fs:
        f.write(str(s) +"\n")
with open("../../models/false_positive/models/process_hypergeometric.pckl", "wb") as f:
    for model in hypergeometric_models:
         pickle.dump(model, f)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
closeness_lgr_proba_clf, closeness_cv, closeness_n_fs, closeness_models = classifiers.multiple_fs_classifier_fp(clf, parameters, process_closeness, process_closeness_test, process_closeness_fs, process_labels_df, jobs=6)
closeness_lgr_proba_clf.to_csv('../../models/false_positive/probability/process_closeness_lgr_proba.csv', index=False)
with open('../../models/false_positive/cv_results/process_closeness.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in closeness_cv))
with open("../../models/false_positive/n_fs/process_closeness.txt", "w") as f:
    for s in closeness_n_fs:
        f.write(str(s) +"\n")
with open("../../models/false_positive/models/process_closeness.pckl", "wb") as f:
    for model in closeness_models:
         pickle.dump(model, f)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
betweenness_lgr_proba_clf, betweenness_cv, betweenness_n_fs, betweenness_models = classifiers.multiple_fs_classifier_fp(clf, parameters, process_betweenness, process_betweenness_test, process_betweenness_fs, process_labels_df, jobs=6)
betweenness_lgr_proba_clf.to_csv('../../models/false_positive/probability/process_betweenness_lgr_proba.csv', index=False)
with open('../../models/false_positive/cv_results/process_betweenness.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in betweenness_cv))
with open("../../models/false_positive/n_fs/process_betweenness.txt", "w") as f:
    for s in betweenness_n_fs:
        f.write(str(s) +"\n")
with open("../../models/false_positive/models/process_betweenness.pckl", "wb") as f:
    for model in betweenness_models:
         pickle.dump(model, f)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
fraction_betweenness_lgr_proba_clf, fraction_betweenness_cv, fraction_betweenness_n_fs, fraction_betweenness_models = classifiers.multiple_fs_classifier_fp(clf, parameters, process_fraction_betweenness, process_fraction_betweenness_test, process_fraction_betweenness_fs, process_labels_df, jobs=6)
fraction_betweenness_lgr_proba_clf.to_csv('../../models/false_positive/probability/process_fraction_betweenness_lgr_proba.csv', index=False)
with open('../../models/false_positive/cv_results/process_fraction_betweenness.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in fraction_betweenness_cv))
with open("../../models/false_positive/n_fs/process_fraction_betweenness.txt", "w") as f:
    for s in fraction_betweenness_n_fs:
        f.write(str(s) +"\n")
with open("../../models/false_positive/models/process_fraction_betweenness.pckl", "wb") as f:
    for model in fraction_betweenness_models:
         pickle.dump(model, f)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
rwr_lgr_proba_clf, rwr_cv, rwr_n_fs, rwr_models = classifiers.multiple_fs_classifier_fp(clf, parameters, process_rwr, process_rwr_test, process_rwr_fs, process_labels_df, jobs=6)
rwr_lgr_proba_clf.to_csv('../../models/false_positive/probability/process_rwr_lgr_proba.csv', index=False)
with open('../../models/false_positive/cv_results/process_rwr.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in rwr_cv))
with open("../../models/false_positive/n_fs/process_rwr.txt", "w") as f:
    for s in rwr_n_fs:
        f.write(str(s) +"\n")
with open("../../models/false_positive/models/process_rwr.pckl", "wb") as f:
    for model in rwr_models:
         pickle.dump(model, f)

### 2.1.2. Disease Steiner Tree

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
disease_hypergeometric_lgr_proba_clf, disease_hypergeometric_cv, disease_hypergeometric_n_fs, disease_hypergeometric_models = classifiers.multiple_fs_classifier_fp(clf, parameters, disease_hypergeometric, disease_hypergeometric_test, disease_hypergeometric_fs, disgenet_labels_df, jobs=6)
disease_hypergeometric_lgr_proba_clf.to_csv('../../models/false_positive/probability/disease_hypergeometric_lgr_proba.csv', index=False)
with open('../../models/false_positive/cv_results/disease_hypergeometric.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in disease_hypergeometric_cv))
with open("../../models/false_positive/n_fs/disease_hypergeometric.txt", "w") as f:
    for s in disease_hypergeometric_n_fs:
        f.write(str(s) +"\n")
with open("../../models/false_positive/models/disease_hypergeometric.pckl", "wb") as f:
    for model in disease_hypergeometric_models:
         pickle.dump(model, f)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]},  
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
disease_closeness_lgr_proba_clf, disease_closeness_cv, disease_closeness_n_fs, disease_closeness_models = classifiers.multiple_fs_classifier_fp(clf, parameters, disease_closeness, disease_closeness_test, disease_closeness_fs, disgenet_labels_df, jobs=6)
disease_closeness_lgr_proba_clf.to_csv('../../models/false_positive/probability/disease_closeness_lgr_proba.csv', index=False)
with open('../../models/false_positive/cv_results/disease_closeness.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in disease_closeness_cv))
with open("../../models/false_positive/n_fs/disease_closeness.txt", "w") as f:
    for s in disease_closeness_n_fs:
        f.write(str(s) +"\n")
with open("../../models/false_positive/models/disease_closeness.pckl", "wb") as f:
    for model in disease_closeness_models:
         pickle.dump(model, f) 

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
disease_betweenness_lgr_proba_clf, disease_betweenness_cv, disease_betweenness_n_fs, disease_betweenness_models = classifiers.multiple_fs_classifier_fp(clf, parameters, disease_betweenness, disease_betweenness_test, disease_betweenness_fs, disgenet_labels_df, jobs=6)
disease_betweenness_lgr_proba_clf.to_csv('../../models/false_positive/probability/disease_betweenness_lgr_proba.csv', index=False)
with open('../../models/false_positive/cv_results/disease_betweenness.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in disease_betweenness_cv))
with open("../../models/false_positive/n_fs/disease_betweenness.txt", "w") as f:
    for s in disease_betweenness_n_fs:
        f.write(str(s) +"\n")
with open("../../models/false_positive/models/disease_betweenness.pckl", "wb") as f:
    for model in disease_betweenness_models:
         pickle.dump(model, f)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
disease_fraction_betweenness_lgr_proba_clf, disease_fraction_betweenness_cv, disease_fraction_betweenness_n_fs, disease_fraction_betweenness_models = classifiers.multiple_fs_classifier_fp(clf, parameters, disease_fraction_betweenness, disease_fraction_betweenness_test, disease_fraction_betweenness_fs, disgenet_labels_df, jobs=6)
disease_fraction_betweenness_lgr_proba_clf.to_csv('../../models/false_positive/probability/disease_fraction_betweenness_lgr_proba.csv', index=False)
with open('../../models/false_positive/cv_results/disease_fraction_betweenness.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in disease_fraction_betweenness_cv))
with open("../../models/false_positive/n_fs/disease_fraction_betweenness.txt", "w") as f:
    for s in disease_fraction_betweenness_n_fs:
        f.write(str(s) +"\n")
with open("../../models/false_positive/models/disease_fraction_betweenness.pckl", "wb") as f:
    for model in disease_fraction_betweenness_models:
         pickle.dump(model, f)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
rwr_lgr_proba_clf, rwr_cv, rwr_n_fs, rwr_models = classifiers.multiple_fs_classifier_fp(clf, parameters, disease_rwr, disease_rwr_test, disease_rwr_fs, disgenet_labels_df, jobs=6)
rwr_lgr_proba_clf.to_csv('../../models/false_positive/probability/disease_rwr_lgr_proba.csv', index=False)
with open('../../models/false_positive/cv_results/disease_rwr.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in rwr_cv))
with open("../../models/false_positive/n_fs/disease_rwr.txt", "w") as f:
    for s in rwr_n_fs:
        f.write(str(s) +"\n")
with open("../../models/false_positive/models/disease_rwr.pckl", "wb") as f:
    for model in rwr_models:
         pickle.dump(model, f)

### 2.1.3. Disease Conservative Module

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
hypergeometric_lgr_proba_clf_conservative, hypergeometric_cv_conservative, hypergeometric_n_fs_conservative, hypergeometric_conservative_models = classifiers.multiple_fs_classifier_fp(clf, parameters, disease_hypergeometric_conservative, disease_conservative_hypergeometric_test, disease_conservative_hypergeometric_fs, disgenet_labels_conservative_df, jobs=6)
hypergeometric_lgr_proba_clf_conservative.to_csv('../../models/false_positive/probability/disease_hypergeometric_lgr_proba_conservative.csv', index=False)
with open('../../models/false_positive/cv_results/disease_hypergeometric_conservative.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in hypergeometric_cv_conservative))
with open("../../models/false_positive/n_fs/disease_hypergeometric_conservative.txt", "w") as f:
    for s in hypergeometric_n_fs_conservative:
        f.write(str(s) +"\n")
with open("../../models/false_positive/models/disease_hypergeometric_conservative.pckl", "wb") as f:
    for model in hypergeometric_conservative_models:
         pickle.dump(model, f)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
closeness_lgr_proba_clf_conservative, closeness_cv_conservative, closeness_n_fs_conservative, closeness_conservative_models = classifiers.multiple_fs_classifier_fp(clf, parameters, disease_closeness_conservative, disease_conservative_closeness_test, disease_conservative_closeness_fs, disgenet_labels_conservative_df, jobs=6)
closeness_lgr_proba_clf_conservative.to_csv('../../models/false_positive/probability/disease_closeness_lgr_proba_conservative.csv', index=False)
with open('../../models/false_positive/cv_results/disease_closeness_conservative.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in closeness_cv_conservative))
with open("../../models/false_positive/n_fs/disease_closeness_conservative.txt", "w") as f:
    for s in closeness_n_fs_conservative:
        f.write(str(s) +"\n")
with open("../../models/false_positive/models/disease_closeness_conservative.pckl", "wb") as f:
    for model in closeness_conservative_models:
         pickle.dump(model, f)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
betweenness_lgr_proba_clf_conservative, betweenness_cv_conservative, betweenness_n_fs_conservative, betweenness_conservative_models = classifiers.multiple_fs_classifier_fp(clf, parameters, disease_betweenness_conservative, disease_conservative_betweenness_test, disease_conservative_betweenness_fs, disgenet_labels_conservative_df, jobs=6)
betweenness_lgr_proba_clf_conservative.to_csv('../../models/false_positive/probability/disease_betweenness_lgr_proba_conservative.csv', index=False)
with open('../../models/false_positive/cv_results/disease_betweenness_conservative.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in betweenness_cv_conservative))
with open("../../models/false_positive/n_fs/disease_betweenness_conservative.txt", "w") as f:
    for s in betweenness_n_fs_conservative:
        f.write(str(s) +"\n")
with open("../../models/false_positive/models/disease_betweenness_conservative.pckl", "wb") as f:
    for model in betweenness_conservative_models:
         pickle.dump(model, f)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
fraction_betweenness_lgr_proba_clf_conservative, fraction_betweenness_cv_conservative, fraction_betweenness_n_fs_conservative, fraction_betweenness_conservative_models = classifiers.multiple_fs_classifier_fp(clf, parameters, disease_fraction_betweenness_conservative, disease_conservative_fraction_betweenness_test, disease_conservative_fraction_betweenness_fs, disgenet_labels_conservative_df, jobs=6)
fraction_betweenness_lgr_proba_clf_conservative.to_csv('../../models/false_positive/probability/disease_fraction_betweenness_lgr_proba_conservative.csv', index=False)
with open('../../models/false_positive/cv_results/disease_fraction_betweenness_conservative.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in fraction_betweenness_cv_conservative))
with open("../../models/false_positive/n_fs/disease_fraction_betweenness_conservative.txt", "w") as f:
    for s in fraction_betweenness_n_fs_conservative:
        f.write(str(s) +"\n")
with open("../../models/false_positive/models/disease_fraction_betweenness_conservative.pckl", "wb") as f:
    for model in fraction_betweenness_conservative_models:
         pickle.dump(model, f)

In [ ]:
importlib.reload(classifiers)
clf = LogisticRegression(random_state=22)
parameters = [{'penalty':['l1','l2'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['liblinear'], 'max_iter':[10, 50, 100]}, 
              {'penalty':['l2', 'none'], 'C':[100, 10, 1.0, 0.1, 0.01],
             'solver': ['sag', 'saga', 'newton-cg'], 'max_iter':[10, 50, 100]}]
rwr_lgr_proba_clf_conservative, rwr_cv_conservative, rwr_n_fs_conservative, rwr_conservative_models = classifiers.multiple_fs_classifier_fp(clf, parameters, disease_rwr_conservative, disease_conservative_rwr_test, disease_conservative_rwr_fs, disgenet_labels_conservative_df, jobs=6)
rwr_lgr_proba_clf_conservative.to_csv('../../models/false_positive/probability/disease_rwr_lgr_proba_conservative.csv', index=False)
with open('../../models/false_positive/cv_results/disease_rwr_conservative.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in rwr_cv_conservative))
with open("../../models/false_positive/n_fs/disease_rwr_conservative.txt", "w") as f:
    for s in rwr_n_fs_conservative:
        f.write(str(s) +"\n")
with open("../../models/false_positive/models/disease_rwr_conservative.pckl", "wb") as f:
    for model in rwr_conservative_models:
         pickle.dump(model, f)